In [7]:
import pandas as pd
import numpy as np

In [8]:
df = pd.read_csv("../data/TIGPSw1_s.csv")

C:\Users\as123\AppData\Local\Temp\ipykernel_18868\2047782468.py:1: DtypeWarning: Columns (19,29,152,324,340,368,383,481,483,505,507,529,531,553,555,577,579,603,627) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/TIGPSw1_s.csv")


In [9]:
# -*- coding: utf-8 -*-
# 人工建立的變項 ID 到變項說明映射字典 (id_map)
# 請根據 TIGPSw1_cb_s.pdf Codebook 文件手動補充與核對

id_map = {
    # --- Metadata / ID Variables ---
    "qb_code": "題本編號",
    "q_name": "問卷名稱",
    "nstudent_id": "學生 ID", # Codebook 寫 nstudent id，您提供的 CSV 標頭是 nstudent_id
    "nschool_id": "學校 ID", # Codebook 寫 nschool id，您提供的 CSV 標頭是 nschool_id
    "nclass": "班級 (區別同一學校內之受測班級)",
    "school_class": "學校班級 ID (由學校ID與校內班級編碼組成)",
    "school_type": "學校類型",
    "seat_n": "座號",

    # --- (一) 個人與家庭基本資料 ---
    "as1": "s1. 請問你的性別(生理性別)?", # 在說明前加上題號 sX. 方便對照
    "as2": "s2. 請問你是哪一年出生?", # (數值，無需值標籤轉換，但可加說明)
    "as3": "s3. 請問你親生父母的婚姻狀態?",
    "aks3": "s3. 請問你親生父母的婚姻狀態?(11)其他，請說明", # 對應的文字欄位
    "as4": "s4. 請問你的主要照顧者（家長）的一些情形",
    "as4a1": "s4a(一). 請問你的第一位主要照顧者是你的?",
    "aks4a1": "s4a(一). 請問你的第一位主要照顧者是你的?(07)其他，請說明",
    "as4b1": "s4b(一). 請問你的第一位主要照顧者 民國出生年?",
    "as4c1": "s4c(一). 請問你的第一位主要照顧者是哪裡人?",
    "aks4c1": "s4c(一). 請問你的第一位主要照顧者是哪裡人?(08)其他，請說明",
    "as4c1_4": "s4c(一). A 請問你的第一位主要照顧者是哪一族?", # 原住民細項
    "as4c1_7": "s4c(一). B 請問你的第一位主要照顧者是哪一國?", # 其他國籍細項
    "aks4c1_7": "s4c(一). B 請問你的第一位主要照顧者是哪一國?(13)其他，請說明",
    "as4d1": "s4d(一). 請問你的第一位主要照顧者的最高學歷?",
    "as4e1": "s4e(一). 請問你的第一位主要照顧者的健康情形?",
    "as4a2": "s4a(二). 請問你的第二位主要照顧者是你的?", # 第二照顧者開始
    "aks4a2": "s4a(二). 請問你的第二位主要照顧者是你的?(07)其他,請說明",
    "as4b2": "s4b(二). 請問你的第二位主要照顧者 民國出生年?",
    "as4c2": "s4c(二). 請問你的第二位主要照顧者是哪裡人?",
    "aks4c2": "s4c(二). 請問你的第二位主要照顧者是哪裡人？(08)其他，請說明",
    "as4c2_4": "s4c(二). 請問你的第二位主要照顧者是哪一族？",
    "as4c2_7": "s4c(二). 請問你的第二位主要照顧者是哪一國？",
    "aks4c2_7": "s4c(二). 請問你的第二位主要照顧者是哪一國？(13)其他，請說明",
    "as4d2": "s4d(二). 請問你的第二位主要照顧者的最高學歷?",
    "as4e2": "s4e(二). 請問你的第二位主要照顧者的健康情形?",
    "as5": "s5. 請問你是否有兄弟姐妹?",
    "as6": "s6. 那你有幾個兄弟姐妹",
    "as6a": "s6a. 請問你的兄弟姊妹的人數: 兄 __ 人", # Codebook 寫 asba，您提供的 CSV 是 as6a
    "as6b": "s6b. 請問你的兄弟姊妹的人數: 姐 __ 人",
    "as6c": "s6c. 請問你的兄弟姊妹的人數: 弟 __ 人",
    "as6d": "s6d. 請問你的兄弟姊妹的人數: 妹 __ 人",
    "as7": "s7. 以下描述符不符合你與兄弟姊妹之間的相處？(B卷)",
    "as7a": "s7a. 我與兄弟姊妹之間相處融洽,會相互照顧與分享。(B卷)", # B卷作答
    "as7b": "s7b. 我與兄弟姊妹之間常有衝突。(B卷)",
    "as7c": "s7c. 我與兄弟姊妹之間會彼此互相比較。(B卷)",
    "as8": "s8. 你目前與下列哪些人同住？",
    "as8_1": "s8. 親生父親", # 多選題拆分
    "as8_2": "s8. 親生母親",
    "as8_3": "s8. 養父/繼父",
    "as8_4": "s8. 養母/繼母",
    "as8_5": "s8. 已婚兄弟",
    "as8_6": "s8. 已婚姊妹",
    "as8_7": "s8. 未婚兄弟",
    "as8_8": "s8. 未婚姊妹",
    "as8_9": "s8. 兄弟的配偶",
    "as8_10": "s8. 姊妹的配偶",
    "as8_11": "s8. 祖父",
    "as8_12": "s8. 祖母",
    "as8_13": "s8. 外祖父",
    "as8_14": "s8. 外祖母",
    "as8_15": "s8. 伯父、伯母",
    "as8_16": "s8. 叔叔、嬸嬸",
    "as8_17": "s8. 姑姑、姑丈",
    "as8_18": "s8. 舅舅、舅媽",
    "as8_19": "s8. 阿姨、姨丈",
    "as8_20": "s8. 表／堂兄姊",
    "as8_21": "s8. 表／堂弟妹",
    "as8_22": "s8. 看護或幫傭",
    "as8_23": "s8. 其他",
    "aks8_23": "s8. 其他，（請說明）",
    "as9": "s9. 下列關於你家中生活的情形，你認為符不符合？",
    "as9a": "s9a. 作決定時,家人會彼此商量。", # Likert scale
    "as9b": "s9b. 家人喜歡共度休閒時光。",
    "as9c": "s9c. 當有家庭活動時,我們家每個人都會參加。",
    "as9d": "s9d. 家人會接納彼此的朋友。",
    "as9e": "s9e. 我受到挫折時,總可以從我的家人那裡得到安慰。",
    "as9f": "s9f. 當我需要幫忙或忠告時,我可以依賴我的家人。",
    "as10": "s10. 請問你認為家裡的經濟狀況為何?",
    "as11": "s11. 請問您目前有沒有宗教信仰?",
    "aks11_7": "s11. 請問教派是? (基督教)", # 稍微整理說明
    "aks11_9": "s11. 請問您目前有沒有宗教信仰?(09)其他,請說明",
    "as12": "s12. 過去一年，你家長和你相處情形如何？(B卷)",
    "as12a": "s12a. 過去一年,家長和你相處情形: 批評你或指責你。(B卷)",
    "as12b": "s12b. 過去一年,家長和你相處情形: 理解你、肯定你的想法。(B卷)",
    "as12c": "s12c. 過去一年,家長和你相處情形: 關心你。(B卷)",
    "as12d": "s12d. 過去一年,家長和你相處情形: 提供解決問題的經驗，給你參考。(B卷)",
    "as12e": "s12e. 過去一年,家長和你相處情形: 對你不聞不問。(B卷)",
    "as12f": "s12f. 過去一年,家長和你相處情形: 跟你常有衝突，彼此關係緊張。(B卷)",
    "as12g": "s12g. 過去一年,家長和你相處情形: 知道你每天的行蹤。(B卷)",
    "as12h": "s12h. 過去一年,家長和你相處情形: 同樣一件事，他們有時處罰小孩，有時不處罰。(B卷)",
    "as12i": "s12i. 過去一年,家長和你相處情形: 處罰你時會打你。(B卷)",
    "as12j": "s12j. 過去一年,家長和你相處情形: 有關你的事，會告訴你他們的想法。(B卷)",
    # --- (二) 學校生活與學業 ---
    "as13": "s13. 請問你對你目前的學校生活滿意嗎?",
    "as14": "s14. 下列是有關學校的描述，你同不同意？",
    "as14a": "s14a. 我喜歡學校。",
    "as14b": "s14b. 我不認為我是學校的一份子。",
    "as14c": "s14c. 我以我的學校為榮。",
    "as15": "s15. 以下對你學校的描述你覺得符不符合？",
    "as15a": "s15a. 學生逃學或翹課情況嚴重。",
    "as15b": "s15b. 學生抽菸或喝酒問題嚴重。",
    "as16": "s16. 下列有關你們班上的說法，你認為符不符合目前的狀況？",
    "as16a": "s16a. 當我需要他們的時候，我的同學們總是會適時伸出援手。",
    "as16b": "s16b. 我不喜歡與班上同學來往。",
    "as16c": "s16c. 當班上的同學需要我的時候，我總是會適時伸出援手。",
    "as16d": "s16d. 我們班的同學相親相愛，猶如一家人。",
    "as16e": "s16e. 與其他班級比較，我們班的同學彼此之間，最互相信賴。",
    "as17": "s17. 請問你的座號是？___號 (A卷)",
    "as18": "s18. 以下要了解你與班上同學在網路上以及實際生活中的相處狀況，請選出你最喜歡與最不喜歡的同學座號，最多可以選擇五位。 (A卷)",
    "as18a": "s18a. 在網路上，你喜歡跟班上哪些同學互動（例如一起打遊戲、講電話、聊天等）？",
    "as18a1": "s18a1. (1)",
    "as18a2": "s18a2. (2)",
    "as18a3": "s18a3. (3)",
    "as18a4": "s18a4. (4)",
    "as18a5": "s18a5. (5)",
    "as18b": "s18b. 在網路上，你不喜歡跟班上哪些同學互動？",
    "as18b1": "s18b1. (1)",
    "as18b2": "s18b2. (2)",
    "as18b3": "s18b3. (3)",
    "as18b4": "s18b4. (4)",
    "as18b5": "s18b5. (5)",
    "as18c": "s18c. 在實際生活（非網路）中，你喜歡跟班上哪些同學互動（例如分組同組、一起吃飯、討論功課、聊心事等）？",
    "as18c1": "s18c1. (1)",
    "as18c2": "s18c2. (2)",
    "as18c3": "s18c3. (3)",
    "as18c4": "s18c4. (4)",
    "as18c5": "s18c5. (5)",
    "as18d": "s18d. 在實際生活（非網路）中，你不喜歡跟班上哪位同學互動？",
    "as18d1": "s18b1. (1)",
    "as18d2": "s18b2. (2)",
    "as18d3": "s18b3. (3)",
    "as18d4": "s18b4. (4)",
    "as18d5": "s18b5. (5)",
    "as19": "s19. 你跟得上學校課業進度嗎？",
    "as20": "s20. 你上學期的平均成績大約如何？",
    "as21": "s21. 家長對你的學業成績有些甚麼要求？",
    "aks21": "s21. 家長對你的學業成績有些甚麼要求？(05)其他，請說明",
    "as22": "s22. 上七年級以來，你有沒有參加校外補習（包含家教與家教班）？",
    "as22a": "s22a. 你補哪幾科？",
    "as22a1": "s22a1. 數學",
    "as22a2": "s22a2. 英語",
    "as22a3": "s22a3. 生物",
    "as22a4": "s22a4. 國文",
    "as22a5": "s22a5. 社會（史、地、公）",
    "as22a6": "s22a6. 資訊/電腦",
    "as22a7": "s22a7. 其他",
    "aks22a7": "s22a7. 其他（請說明）",
    "as23": "s23. 請問父母（家長）對你下列未來發展的期望程度如何？請從1 到10 分點選期望程度。",
    "as23a": "s23a. 他們期望我的學業成就可以令他們引以自豪(B卷)",
    "as23b": "s23b. 他們期望我將來可以有一份高薪的工作(B卷)",
    "as23c": "s23c. 他們期望我將來可以分擔家庭的經濟(B卷)",
    "as23d": "s23d. 他們期望我可以光宗耀祖(B卷)",
    "as24": "s24. 請問以下敘述是否符合你家長的情形？(B卷)",
    "as24a": "s24a. 家長知道我在學校主要的授課老師是誰。(B卷)",
    "as24b": "s24b. 家長知道我在學校的學習狀況（例如: 是否跟得上進度？）。(B卷)",
    "as24c": "s24c. 家長知道我在學校的課業內容（例如:現在教到哪裡？）。(B卷)",
    "as24d": "s24d. 家長會參加學校的家長會/學校日。(B卷)",
    "as24e": "s24e. 家長會為班上的活動或設備出錢出力。(B卷)",
    "as24f": "s24f. 家長會為班上的活動或設備出錢出力。(B卷)",
    "as24g": "s24g. 家長會花時間在我的學校功課上。(B卷)",
    "as24h": "s24h. 家長會幫我看聯絡簿。(B卷)",
    "as24i": "s24i. 家長會幫我買課本以外的評量、測驗卷等補充教材。(B卷)",
    "as24j": "s24j. 家長會上網幫我搜尋相關的教學材料或教學平台。(B卷)",
    "as25": "s25. 請問你們班有沒有下列群組（LINE 或臉書等群組）？(B卷)",
    "as25a": "s25a. 班上同學的群組，不包含導師。(B卷)",
    "as25b": "s25b. 班上同學的群組，導師有加入。(B卷)",
    "as26": "s26. 上七年級以來，你的導師是否曾主動聯絡你的家長？(B卷)",
    "as26a": "s26a. 導師多常聯絡你的家長？(B卷)",
    "as26b": "s26b. 導師通常是連絡哪位家長？(B卷)",
    "aks26b": "s26b. 導師通常是連絡哪位家長？(5)其他家長（請說明）(B卷)",
    "as27": "s27. 上七年級以來，你的家長是否曾主動聯絡導師？(B卷)",
    "as27a": "s27a.他們多常聯絡你的導師？(B卷)",
    "as27b": "s27b. 通常是誰連絡你的導師？(B卷)",
    "aks27b": "s27b. 通常是誰連絡你的導師？(5)其他家長（請說明）(B卷)",
    # --- (三) 網路生活經驗 ---
    "as28": "s28. 關於上網，你父親和你有沒有以下的情形？(B卷)",
    "as28a": "s28a. 會規定你什麼時候可以上網，什麼時候不行。(B卷)",
    "as28b": "s28b. 會禁止你上某些網站。(B卷)",
    "as28c": "s28c. 會限制你上網時間的長短。(B卷)",
    "as28d": "s28d. 會和你一起聊網路上發生的趣聞或事件。(B卷)",
    "as28e": "s28e. 會和你一起上網玩遊戲或追劇。(B卷)",
    "as28f": "s28f. 會陪你一起上網查資料或做作業。(B卷)",
    "as28g": "s28g. 會鼓勵你上網。(B卷)",
    "as28h": "s28h. 會和你討論使用網路的情形和心得。(B卷)",
    "as28i": "s28i. 認為擅長使用網路對你的未來很有幫助。(B卷)",
    "as28j": "s28j. 擔心你上網會產生健康問題（如：眼睛、體能）。(B卷)",
    "as28k": "s28k. 會擔心你面臨的網路風險（如：負面訊息、色情、霸凌、詐騙等）。 (B卷)",
    "as28l": "s28l. 擔心你的課業成績會因為上網而退步。(B卷)",
    "as28m": "s28m. 會問你在網上做什麼。(B卷)",
    "as28n": "s28n. 知道你在網路世界裡的朋友是誰。(B卷)",
    "as28o": "s28o. 會查看你的網頁瀏覽記錄。(B卷)",
    "as28p": "s28p. 會在你上網時碎碎念。(B卷)",
    "as28q": "s28q. 會因為你上網而責罵你。(B卷)",
    "as28r": "s28r. 會因為你在網路購物而和你起衝突。(B卷)",
    "as29": "s29. 關於上網，你母親和你有沒有以下的情形？(B卷)",
    "as29a": "s29a. 會規定你什麼時候可以上網，什麼時候不行。(B卷)",
    "as29b": "s29b. 會禁止你上某些網站。(B卷)",
    "as29c": "s29c. 會限制你上網時間的長短。(B卷)",
    "as29d": "s29d. 會和你一起聊網路上發生的趣聞或事件。(B卷)",
    "as29e": "s29e. 會和你一起上網玩遊戲或追劇。(B卷)",
    "as29f": "s29f. 會陪你一起上網查資料或做作業。(B卷)",
    "as29g": "s29g. 會鼓勵你上網。(B卷)",
    "as29h": "s29h. 會和你討論使用網路的情形和心得。(B卷)",
    "as29i": "s29i. 認為擅長使用網路對你的未來很有幫助。(B卷)",
    "as29j": "s29j. 擔心你上網會產生健康問題（如：眼睛、體能）。(B卷)",
    "as29k": "s29k. 會擔心你面臨的網路風險（如：負面訊息、色情、霸凌、詐騙等）。 (B卷)",
    "as29l": "s29l. 擔心你的課業成績會因為上網而退步。(B卷)",
    "as29m": "s29m. 會問你在網上做什麼。(B卷)",
    "as29n": "s29n. 知道你在網路世界裡的朋友是誰。(B卷)",
    "as29o": "s29o. 會查看你的網頁瀏覽記錄。(B卷)",
    "as29p": "s29p. 會在你上網時碎碎念。(B卷)",
    "as29q": "s29q. 會因為你上網而責罵你。(B卷)",
    "as29r": "s29r. 會因為你在網路購物而和你起衝突。(B卷)",
    "as30": "s30. 關於你上網，主要是誰在管？(B卷)",
    "aks30": "s30. 關於你上網，主要是誰在管？(9)其他人（請說明）(B卷)",
    "as31": "s31. 你有沒有社群媒體帳號？(B卷)",
    "as31a1": "s31a. 有哪些社群媒體的帳號？: LINE (B卷)",
    "as31a2": "s31a. 有哪些社群媒體的帳號？: Facebook (B卷)",
    "as31a3": "s31a. 有哪些社群媒體的帳號？: Instagram (B卷)",
    "as31a4": "s31a. 有哪些社群媒體的帳號？: WeChat  (B卷)",
    "as31a5": "s31a. 有哪些社群媒體的帳號？: Dcard (B卷)",
    "as31a6": "s31a. 有哪些社群媒體的帳號？: 抖音（Tiktok） (B卷)",
    "as31a7": "s31a. 有哪些社群媒體的帳號？: 小紅書 (B卷)",
    "as31a8": "s31a. 有哪些社群媒體的帳號？: 批踢踢（PTT） (B卷)",
    "as31a9": "s31a. 有哪些社群媒體的帳號？: Discord (B卷)",
    "as31a10": "s31a. 有哪些社群媒體的帳號？: 其他（請說明） (B卷)",
    "aks31a10": "s31a. 有哪些社群媒體的帳號？(10)其他（請說明）: 其他（請說明） (B卷)",
    "as32": "s32. 請問你和家長（父母）的即時通訊情形：（父母合併為：家長（父母））",
    "as32a": "s32a. 你平常和他們使用即時通訊的頻率如何？(B卷)",
    "as32b": "s32b. 你滿不滿意和他們的即時通訊情形？(B卷)",
    "as32c": "s32c. 對於你和他們的即時通訊，符不符合以下情形？(B卷)",
    "as32c1": "s32c1. 增加彼此聯繫與訊息傳達。(B卷)",
    "as32c2": "s32c2. 增加彼此關心表達與心事分享。(B卷)",
    "as32c3": "s32c3. 減少彼此面對面互動。(B卷)",
    "as32c4": "s32c4. 引起彼此的衝突或誤解。(B卷)",
    "as33": "s33. 請問你和你兄弟姊妹間的即時通訊情形：(B卷)",
    "as33a": "s33a. 你平常和他們使用即時通訊的頻率如何？(B卷)",
    "as33b": "s33b. 你滿不滿意和他們的即時通訊情形？(B卷)",
    "as33c": "s33c. 對於你和他們的即時通訊，符不符合以下情形？(B卷)",
    "as33c1": "s33c1. 增加彼此聯繫與訊息傳達。(B卷)",
    "as33c2": "s33c2. 增加彼此關心表達與心事分享。(B卷)",
    "as33c3": "s33c3. 減少彼此面對面互動。(B卷)",
    "as33c4": "s33c4. 引起彼此的衝突或誤解。(B卷)",
    "as34": "s34. 在平日（上學日），請問你會不會上網？(A卷)",
    "as35": "s35. 在平日（上學日），請問你每天花多少時間上網做以下活動？(A卷)",
    "as35a": "s35a. 完成學校功課（查找完成作業需要的資料）(A卷)",
    "as35b": "s35b. 課外的學習（各種線上付費或免費的課程）(A卷)",
    "as35c": "s35c. 玩線上遊戲(A卷)",
    "as35d": "s35d. 看影片、聽音樂、迷因梗圖、卡通、漫畫(A卷)",
    "as35e": "s35e. 和他人聊天（傳訊息）(A卷)",
    "as35f": "s35f. 在網路上瀏覽自己有興趣的資訊(A卷)",
    "as36": "s36. 請問你使用網路搜尋過哪些健康議題？【可複選】",
    "as36_1": "s36. 皮膚健康", # 多選題拆分
    "as36_2": "s36. 身材體態",
    "as36_3": "s36. 營養",
    "as36_4": "s36. 性教育",
    "as36_5": "s36. 憂鬱",
    "as36_6": "s36. 焦慮",
    "as36_7": "s36. 壓力調適",
    "as36_8": "s36. 其他（請說明）",
    "aks36_8": "s36. 請問你使用網路搜尋過哪些健康議題？(10)其他（請說明）",
    "as36_9": "s36. 都沒有",
    "as37": "s37. 當你在現實生活遇到困難感到憂慮時，你會上網： (A卷)",
    "as37a": "s37a. 找人聊天給你支持或安慰。 (A卷)",
    "as37b": "s37b. 找人討論問題，設法解決困難。 (A卷)",
    "as37c": "s37c. 搜尋相關的資訊，設法解決困難。(A卷)",
    "as37d": "s37d. 瀏覽跟困難無關的資訊，轉移自己的注意力。(A卷)",
    "as37e": "s37e. 玩遊戲來減少困擾或壓力感受。(A卷)",
    "as37f": "s37f. 購物來改善心情或壓力感受。(A卷)",
    "as38": "s38. 以下描述符不符合你的個人經驗感受呢？(A卷)",
    "as38a": "s38a. 我喜歡自己在網路上的形象。(A卷)",
    "as38b": "s38b. 我在網路上可以表現出我最好的一面。(A卷)",
    "as38c": "s38c. 在網路世界中我覺得自己可以成為理想的自己。(A卷)",
    "as38d": "s38d. 我喜歡自己在現實生活裡的形象。(A卷)",
    "as38e": "s38e. 我在現實生活裡可以表現出我最好的一面。(A卷)",
    "as38f": "s38f. 在現實生活裡我覺得自己可以成為理想的自己。(A卷)",
    "as38g": "s38g. 我在網路世界裡經常會扮演成不一樣的角色。 (A卷)",
    "as38h": "s38h. 我在不同的社群媒體中（如臉書、IG）會有不一樣的形象。(A卷)",
    "as38i": "s38i. 我喜歡在網路上擁有多種不一樣的身分、形象或樣貌。(A卷)",
    "as38j": "s38j. 我在網路上呈現自我的方式，與在現實生活的自我並不相同。(A卷)",
    "as38k": "s38k. 我覺得我在網絡世界和現實世界中是不同的人。(A卷)",
    "as38l": "s38l. 我在網路世界中表現出與現實世界中截然不同的言行舉止。(A卷)",
    "as38m": "s38m. 比起在現實生活，我覺得自己在網路上比較輕鬆自在(A卷)",
    "as38n": "s38n. 我喜歡在網路上的生活，勝過於現實中的生活。(A卷)",
    "as38o": "s38o. 如果可以，我希望自己可以一直待在網路上。(A卷)",
    "as39": "s39. 你同不同意以下說法？(A卷)",
    "as39a": "s39a. 沒跟上朋友們在網路上聊天或開玩笑的話題，會讓我感到很焦慮。 (A卷)",
    "as39b": "s39b. 錯過和朋友們在網路上的聚會，會讓我感到很困擾。(A卷)",
    "as39c": "s39c. 我會隨時透過網路追蹤朋友們的行蹤和做的事情。 (A卷)",
    "as39d": "s39d. 沒跟上網路上的首播、直播或活動，我會感到很困擾。(A卷)",
    "as40": "s40. 關於你在網路上的意見表達，以下敘述你同不同意？(A卷)",
    "as40a": "s40a. 我經常會在網路上和他人比較自己在生活上獲得的成就。 (A卷)",
    "as40b": "s40b. 我總是在網路上留意他人的表現，並和自己的表現進行比較。(A卷)",
    "as40c": "s40c. 我經常在網路上和他人比較自己受歡迎的程度。(A卷)",
    "as40d": "s40d. 如果我想要更瞭解某些事情，我會試著去網路上看看別人的想法。(A卷)",
    "as40e": "s40e. 我總是好奇網路上的人們在和我遇到相同的問題時會怎麼想或怎麼做。(A卷)",
    "as40f": "s40f. 我喜歡在網路上看看其他人的不同觀點和經歷。(A卷)",
    "as41": "s41. 關於上網，以下敘述符不符合您的情形？",
    "as41a": "s41a. 想上網而無法上網的時候，我就會感到坐立不安。",
    "as41b": "s41b. 只要有一段時間沒有上網，我就會覺得心裡不舒服。",
    "as41c": "s41c. 我只要超過一天沒上網玩，就會很難受。",
    "as41d": "s41d. 發現自己上網休閒的時間越來越長。",
    "as41e": "s41e. 平均而言，我每週上網休閒的時間比以前增加許多。",
    "as41f": "s41f. 我實際上網所花的時間遠遠超過原來預期的時間。",
    "as41g": "s41g. 上網對我的學業或工作已造成一些負面的影響。",
    "as41h": "s41h. 上網已經很明顯地危害到我的生理或心理健康。",
    "as41i": "s41i. 因為上網的關係，我和家人及朋友的互動減少了。",
    "as41j": "s41j. 我常因熬夜上網休閒，導致白天精神或體力很差。",
    # --- (四) 同儕關係與霸凌行為 ---
    "as42": "s42. 以下有關於你最要好的朋友的描述，你同不同意？",
    "as42a": "s42a. 當我需要幫助或忠告時，我 (1)很不同意可以依賴他（們）。",
    "as42b": "s42b. 他（們）有時候會惹事生非。",
    "as42c": "s42c. 他（們）關心我。",
    "as42d": "s42d. 我常會得到他（們）的幫忙。 ",
    "as42e": "s42e. 我受到挫折時，總可以從他（們）那裡得到安慰。",
    "as43": "s43. 請問你認為你周遭的朋友是可以信任的，還是與他們來往時要儘量小心？(A卷)",
    "as44": "s44. 你認為你周遭朋友只要有機會就會想要占你便宜，還是他們常常會公平合理對待你？(A卷)",
    "as45": "s45. 你認為你周遭朋友只要有機會就會想要占你便宜，還是他們常常會公平合理對待你？(A卷)",
    "as46": "s46. 除了你周遭的朋友外，你日常生活中遇到的人是可以信任的，還是與他們來往時要儘量小心？(A卷)",
    "as47": "s47. 除了你周遭的朋友外，你日常生活中遇到的人只要有機會就會想要占你便宜，還是他們常常會公平合理對待你？(A卷)",
    "as48": "s48. 除了你周遭的朋友外，你日常生活中遇到的人大部分時間是樂於助人的，還是他們只顧自己？(A卷)",
    "as49": "s49. 七年級以來，同學之間有時會發生衝突。當有同學被欺負時，請問你多常做以下的事情？(B卷)",
    "as49a": "s49a. 當同學被惡意嘲笑時，我會幫忙被嘲笑的人。(B卷)",
    "as49b": "s49b. 同學被威脅、騷擾時，我會保護被威脅的人。(B卷)",
    "as49c": "s49c. 當同學被惡意排擠而無法分組或參加活動時，我會設法讓同學不要被排擠。(B卷)",
    "as49d": "s49d. 當別人嘲笑特定同學，我不會加入對話，或將這些內容傳給別人。(B卷)",
    "as49e": "s49e. 當我知道有同學被威脅、騷擾，我不會有任何反應或行動。(B卷)",
    "as49f": "s49f. 當有人被惡意排擠而無法分組或參加活動時，我不會多管閒事。(B卷)",
    "as49g": "s49g. 當我收到別人傳來同學的出醜或是不雅照片，我不會傳給別人。(B卷)",
    "as49h": "s49h. 當同學被惡意嘲笑時，我會跟著一起嘲笑那個人。(B卷)",
    "as49i": "s49i. 當同學被威脅、騷擾時，我會跟著做這些行為。(B卷)",
    "as49j": "s49j. 當同學被排擠而無法分組或參加活動時，我也不會跟這個同學一組。(B卷)",
    "as50": "s50. 請問你同不同意以下描述？(B卷)",
    "as50a": "s50a. 我認為霸凌是一種不道德的行為。(B卷)",
    "as50b": "s50b. 我們班很多同學可以容忍霸凌。(B卷)",
    "as51": "s51. 上七年級以來，你曾在實際生活中（非網上），被其他人辱罵、惡意取笑、破壞或搶取東西、肢體攻擊、拒絕加入討論或活動嗎？",
    "as51a": "s51a. 上七年級以來，你曾在實際生活中（非網上）遇到這樣的事情嗎？",
    "as51a1": "s51a1. 別人用髒話罵或威脅我。",
    "as51a2": "s51a2. 別人說我的壞話。",
    "as51a3": "s51a3. 不讓我加入討論、活動或不准別人跟我說話、一起行動。",
    "as51a4": "s51a4. 被嘲笑外表或言行舉止。",
    "as51a5": "s51a5. 被嘲笑我喜歡某個人。",
    "as51a6": "s51a6. 故意破壞我的東西或故意藏起來。",
    "as51a7": "s51a7. 勒索或強要我的東西。",
    "as51a8": "s51a8. 別人對我肢體攻擊。",
    "as51a9": "s51a9. 其他霸凌行為（請說明）",
    "as51b": "s51b. 請問是誰對你這麼做？【可複選】",
    "as51b1": "s51b. 同班同學",
    "as51b2": "s51b. 同校不同班的同學",
    "as51b3": "s51b. 不同校的朋友",
    "as51b4": "s51b. 鄰居",
    "as51b5": "s51b. 其他（請說明）",
    "aks51b5": "s51b. 其他（請說明）",
    "as51c": "s51c. 那你實際上如何反應？【可複選】",
    "as51c1": "s51c. 好好說明澄清",
    "as51c2": "s51c. 要求當事人停止",
    "as51c3": "s51c. 忽略這些人與事情",
    "as51c4": "s51c. 投入其他有趣或有意義的事情",
    "as51c5": "s51c. 因為太困擾，無法做好平日應該要做的事",
    "as51c6": "s51c. 回擊、跟當事人對抗",
    "as51c7": "s51c. 告訴其他同學或朋友",
    "as51c8": "s51c. 告訴家人",
    "as51c9": "s51c. 告訴老師",
    "as51c10": "s51c. 不去會遇到那個人的場所",
    "as51c11": "s51c. 改變郵件帳號、電話號碼或網路封鎖讓當事人無法聯繫到我",
    "as51c12": "s51c. 其他（請說明）",
    "aks51c12": "s51c. 其他（請說明）",
    "as52": "s52. 上七年級以來，你曾在實際生活中（非網上），對別人辱罵、惡意取笑、破壞或搶取東西、肢體攻擊、拒絕讓某個人加入討論或活動嗎？",
    "as52a": "s52a. 上七年級以來，你曾在實際生活中（非網上）對別人做過這樣的事嗎？",
    "as52a1": "s52a1. 用髒話罵或威脅某人。",
    "as52a2": "s52a2. 說某人的壞話。",
    "as52a3": "s52a3. 不讓某人加入活動、討論事情或不准別人跟某人說話、一起行動。",
    "as52a4": "s52a4. 嘲笑某人的外貌或言行舉止。",
    "as52a5": "s52a5. 嘲笑某人他喜歡某個人。",
    "as52a6": "s52a6. 故意破壞某人的東西或故意藏起來。",
    "as52a7": "s52a7. 勒索或強要某人的東西。",
    "as52a8": "s52a8. 對某人肢體攻擊。",
    "as52a9": "s52a9. 其他霸凌行為（請說明",
    "as52b": "s52b. 請問你曾對誰這麼做？【可複選】",
    "as52b1": "s52b. 同班同學",
    "as52b2": "s52b. 同校不同班的同學",
    "as52b3": "s52b. 不同校的朋友",
    "as52b4": "s52b. 鄰居",
    "as52b5": "s52b. 其他（請說明）",
    "aks52b5": "s52b. 其他（請說明）",
    "as53": "s53. 上七年級以來，你曾在網路上，被其他人辱罵、惡意取笑、拒絕加入討論、散布丟臉的影片、發布不實的訊息嗎？",
    "as53a": "s53a. 上七年級以來，你曾在網路上遇到這樣的事情嗎？",
    "as53a1": "s53a1. 別人用髒話罵或威脅我。",
    "as53a2": "s53a2. 別人說我的壞話。",
    "as53a3": "s53a3. 不讓我加入討論。",
    "as53a4": "s53a4. 被嘲笑外表或言行舉止。",
    "as53a5": "s53a5. 被嘲笑我喜歡某個人。",
    "as53a6": "s53a6. 在網路散布使我丟臉的照片或影片。",
    "as53a7": "s53a7. 假冒我的社交帳號或是電子郵件，在網路上發布不實訊息。",
    "as53a8": "s53a8. 其他霸凌行為（請說明）",
    "as53b": "s53b. 請問是誰對你這麼做？【可複選】",
    "as53b1": "s53b. 同班同學",
    "as53b2": "s53b. 同校不同班的同學",
    "as53b3": "s53b. 不同校的朋友",
    "as53b4": "s53b. 鄰居",
    "as53b5": "s53b. 其他（請說明）",
    "aks53b5": "s53b. 其他（請說明）",
    "as53c": "s53c. 那你實際上如何反應？【可複選】",
    "as53c1": "s53c. 好好說明澄清",
    "as53c2": "s53c. 要求當事人停止",
    "as53c3": "s53c. 忽略這些人與事情",
    "as53c4": "s53c. 投入其他有趣或有意義的事情",
    "as53c5": "s53c. 因為太困擾，無法做好平日應該要做的事",
    "as53c6": "s53c. 回擊、跟當事人對抗",
    "as53c7": "s53c. 告訴其他同學或朋友",
    "as53c8": "s53c. 告訴家人",
    "as53c9": "s53c. 告訴老師",
    "as53c10": "s53c. 不去會遇到那個人的場所",
    "as53c11": "s53c. 改變郵件帳號、電話號碼或網路封鎖讓當事人無法聯繫到我",
    "as53c12": "s53c. 其他（請說明）",
    "aks53c12": "s53c. 其他（請說明）",
    "as54": "s54. 上七年級以來，你曾在網路上，對別人辱罵、惡意取笑、拒絕加入討論、散布丟臉的影片、發布不實訊息等事情嗎？",
    "as54a": "s54a. 上七年級以來，你曾在網路上對別人做過這樣的事嗎？",
    "as54a1": "s54a1. 用髒話罵或威脅某人。",
    "as54a2": "s54a2. 說某人的壞話。",
    "as54a3": "s54a3. 不讓某人加入討論。",
    "as54a4": "s54a4. 嘲笑某人的外貌或言行舉止。",
    "as54a5": "s54a5. 嘲笑某人他喜歡某個人。",
    "as54a6": "s54a6. 在網路散布使某人丟臉的照片或影片。",
    "as54a7": "s54a7. 用某人的社交帳號或是電子郵件，發布不實的訊息給其他人。",
    "as54a8": "s54a8. 其他霸凌行為（請說明）",
    "as54b": "s54b. 請問你曾對誰這麼做？【可複選】",
    "as54b1": "s54b. 同班同學",
    "as54b2": "s54b. 同校不同班的同學",
    "as54b3": "s54b. 不同校的朋友",
    "as54b4": "s54b. 鄰居",
    "as54b5": "s54b. 其他（請說明）",
    "aks54b5": "s54b. 其他（請說明）",
    "as55": "s55. 過去6 個月以來，請問你有沒有做過下列的事情？【可複選】",
    "as55_1": "s55. 考試作弊 ", # 多選題拆分
    "as55_2": "s55. 離家出走",
    "as55_3": "s55. 蹺課或逃學",
    "as55_4": "s55. 偷拿別人的東西",
    "as55_5": "s55. 故意破壞別人的東西",
    "as55_6": "s55. 蓄意打傷別人",
    "as55_7": "s55. 壓力調適",
    "as55_8": "s55. 抽紙菸(香菸)",
    "as55_9": "s55. 喝酒",
    "as55_10": "s55. 嚼檳榔",
    "as55_11": "s55. 收聽或觀看盜版的音樂與影片",
    "as55_12": "s55. 下載非法軟體",
    "as55_13": "s55. 深夜不回家在外遊蕩",
    "as55_14": "s55. 都沒有",
    # --- (五) 數位環境與素養 ---
    "as56": "s56. 請問你使用以下這些設備的頻率如何？ (A卷)",
    "as56a": "s56a. 電腦（含桌機或筆電）(A卷)",
    "as56b": "s56b. 智慧型手機(A卷)",
    "as56c": "s56c. 穿戴裝置（Apple  watch、小米手環等）(A卷)",
    "as56d": "s56d. 平板或電子書閱讀器（iPad, Kindle...）(A卷)",
    "as56e": "s56e. 遊 戲 機 （ XBOX, PS5, Switch......）(A卷)",
    "as57": "s57. 請問你有多常會遇到下面的狀況？ (A卷)",
    "as57a": "s57a. 對我來說，要購買 3C 產品（手機、電腦、平板）的費用太高。(A卷)",
    "as57b": "s57b. 我住的地方沒有網路訊號或是訊號太弱。(A卷)",
    "as57c": "s57c. 對我來說，使用網路要付的費用太貴。(A卷)",
    "as57d": "s57d. 對我來說，很難找到一個不受干擾的空間來上網。(A卷)",
    "as58": "s58. 以下有關使用手機或上網的原因，你同不同意？ (A卷)",
    "as58a": "s58a. 是打發時間的好方法。(A卷)",
    "as58b": "s58b. 可以讓我找到和連接我覺得對我中意或磁場相合的人。(A卷)",
    "as58c": "s58c. 提升我在讀書學習或完成任務的表現。(A卷)",
    "as58d": "s58d. 讓我知道運動、演唱會、或各種娛樂活動的最新訊息。(A卷)",
    "as58e": "s58e. 讓我可以分享我的好點子或我做的東西。(A卷)",
    "as59": "s59. 以下敘述符不符合你的上網學習情形？",
    "as59a": "s59a. 讀書或寫作業時，我會先將無關的網站、即時通訊、手機APP 或提醒聲音關掉。",
    "as59b": "s59b. 我能要求自己先完成作業或讀書進度後，才能去看我喜歡的網站或玩手機。",
    "as59c": "s59c. 我會運用學習平台上的儀表板，了解自己的認真或表現情況（如：目前得分、作業缺交、影片觀看記錄、相對於全班的表現等）。",
    "as59d": "s59d. 我會運用學習平台以外的軟體（如：安排時間-Google 日曆、專注-Forrest 、備忘錄-Notion、閃字卡-Anki 等），安排我的學習進度。",
    "as60": "s60. 關於電腦與網路的操作，你使用情形如何？",
    "as60a": "s60a. 我會運用定位服務找到我要的資訊（地圖導航、交通、美食等）。",
    "as60b": "s60b. 我定期更換密碼（手機、平板、電腦、IG、抖音、Dcard等）。",
    "as60c": "s60c. 同學、閨密、或親朋好友知道我的密碼。 ",
    "as61": "s61. 以下關於媒體網站的運用敘述，你同不同意？",
    "as61a": "s61a. 我能判斷適合找到最好答案的關鍵字。",
    "as61b": "s61b. 我能快速找到我曾拜訪過的網站。",
    "as61c": "s61c. 我很清楚自己正在觀看特定網站或使用特定app 的目的。",
    "as61d": "s61d. 我清楚哪些訊息可以在網路上公開分享，哪些不行。",
    "as61e": "s61e. 我能注意上網時給出的評語是不是恰當。",
    "as61f": "s61f. 貼文和傳照片影片時，我會視情況設定要分享給誰看到。",
    "as61g": "s61g. 我能從網路找到其他人的影音，進行二次創作或改編。",
    "as61h": "s61h. 我能根據網頁/ 影音所屬的著作權規範（如創用 cc）來運用找到的素材。",
    "as61i": "s61i. 我能將所製作的影片/網頁/圖片上傳到雲端空間。",
    # --- (六) 生活經驗與態度 ---
    "as62": "s62. 整體來說，你對自己的生活感到滿意嗎？",
    "as63": "s63. 整體來說，你覺得最近的日子過得快樂嗎？",
    "as64": "s64. 整體而言，你覺得自己目前的健康狀況如何？",
    "as65": "s65. 下列是有關你對自己的描述，你同不同意？",
    "as65a": "s65a. 我是個有價值（有用）的人。",
    "as65b": "s65b. 我用積極樂觀的態度看待我自己。",
    "as65c": "s65c. 我很滿意我自己。",
    "as66": "s66. 關於這些描述，你同不同意？(A卷)",
    "as66a": "s66a. 我願意信任他人。",
    "as66b": "s66b. 我相信自己會受到他人的歡迎。",
    "as66c": "s66c. 我可以很自在地讓他人依賴我。",
    "as66d": "s66d. 我會很擔心他人不是真的喜歡我。",
    "as66e": "s66e. 當他人不支持我時，我會對自己感到很難過。",
    "as66f": "s66f. 我會很在意自己與他人之間的人際互動。",
    "as66g": "s66g. 我不願意向他人分享自己的經驗與感受。",
    "as66h": "s66h. 我會希望他人能與我保持距離。",
    "as66i": "s66i. 我會盡量避免與他人過於友好。",
    "as67": "s67. 請問在一般要上學的日子裡，你通常：(A卷)",
    "as67a": "s67a. 幾點到學校？",
    "as67a1": "s67a. ______點",
    "as67a2": "s67a. _______分",
    "as67b": "s67b. 幾點離開學校？",
    "as67b1": "s67b. ______點",
    "as67b2": "s67b. _______分",
    "as68": "s68. 請問你過去一個月來的日常(大多數)睡眠習慣：(A卷)",
    "as68a": "s68a. 過去一個月來，在需要上課的日子，你通常幾點上床睡覺？ ",
    "as68a1": "s68a. ______點",
    "as68a2": "s68a. _______分",
    "as68b": "s68b. 過去一個月來，在需要上課的日子，你通常幾點上床睡覺？ ",
    "as68b1": "s68b. ______點",
    "as68b2": "s68b. _______分",
    "as68c": "s68c. 過去一個月來，在放假前一天或不需要上課的日子，你通常幾點上床睡覺？ ",
    "as68c1": "s68c. ______點",
    "as68c2": "s68c. _______分",
    "as68d": "s68d. 過去一個月來，在不需要上課的日子，你通常幾點起床？",
    "as68d1": "s68d. ______點",
    "as68d2": "s68d. _______分",
    "as68e": "s68e. 過去一個月來，你整體的睡眠品質好不好？",
    "as69": "s69. 過去一個月，請問以下飲料你平均一週七天喝多少？(A卷)",
    "as69a": "s60a. 鋁箔包、利樂包或瓶裝的含糖飲料",
    "as69b": "s69b. 手搖杯",
    "as70": "s70. 請問在一般要上學的日子裡，在下列時間中你會做什麼事情？請從下列的活動列表中選擇活動的號碼填入。一個時段可以填多個活動，請區分主要活動（一個）和次要活動（可以不限一個）。若你做的事情不在列表之中，請選其他，並告訴我們是什麼活動。",
    "as70a": "s70a. 晚上6:00-7:00 你的主要活動是什麼？",
    "aks70a": "s70a. 晚上6:00-7:00 你的主要活動是什麼？(21)其他，請說明",
    "as70b": "s70b. 晚上6:00-7:00 你的次要活動是什麼（可以不限一個）？",
    "as70b1": "s70b. 學校課後輔導/補習/安親班/上家教",
    "as70b2": "s70b. 寫作業/唸書/準備考試",
    "as70b3": "s70b. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70b4": "s70b. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70b5": "s70b. 吃東西或用餐",
    "as70b6": "s70b. 通勤",
    "as70b7": "s70b. 做家事",
    "as70b8": "s70b. 睡覺",
    "as70b9": "s70b. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70b10": "s70b. 照護/看顧家人",
    "as70b11": "s70b. 與家人/朋友/鄰居聊天",
    "as70b12": "s70b. 照顧寵物/與寵物玩",
    "as70b13": "s70b. 看顧家中的生意/農務/工廠",
    "as70b14": "s70b. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70b15": "s70b. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70b16": "s70b. 在網路上與他人聊天（傳訊息）",
    "as70b17": "s70b. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70b18": "s70b. 在網路上直播",
    "as70b19": "s70b. 線上購物",
    "as70b20": "s70b. 外出遊玩/運動",
    "as70b21": "s70b. 其他（請說明）",
    "aks70b21": "s70b. 其他（請說明）",
    "as70c": "s70c. 晚上7:00-8:00 你的主要活動是什麼？",
    "aks70c": "s70c. 晚上7:00-8:00 你的主要活動是什麼？(21)其他，請說明",
    "as70d": "s70d. 晚上7:00-8:00 你的次要活動是什麼(可以不限一個)？",
    "as70d1": "s70d. 學校課後輔導/補習/安親班/上家教",
    "as70d2": "s70d. 寫作業/唸書/準備考試",
    "as70d3": "s70d. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70d4": "s70d. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70d5": "s70d. 吃東西或用餐",
    "as70d6": "s70d. 通勤",
    "as70d7": "s70d. 做家事",
    "as70d8": "s70d. 睡覺",
    "as70d9": "s70d. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70d10": "s70d. 照護/看顧家人",
    "as70d11": "s70d. 與家人/朋友/鄰居聊天",
    "as70d12": "s70d. 照顧寵物/與寵物玩",
    "as70d13": "s70d. 看顧家中的生意/農務/工廠",
    "as70d14": "s70d. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70d15": "s70d. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70d16": "s70d. 在網路上與他人聊天（傳訊息）",
    "as70d17": "s70d. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70d18": "s70d. 在網路上直播",
    "as70d19": "s70d. 線上購物",
    "as70d20": "s70d. 外出遊玩/運動",
    "as70d21": "s70d. 其他（請說明）",
    "aks70d21": "s70d. 其他（請說明）",
    "as70e": "s70e. 晚上8:00-9:00 你的主要活動是什麼？",
    "aks70e": "s70e. 晚上8:00-9:00 你的主要活動是什麼？(21)其他，請說明",
    "as70f": "s70f. 晚上8:00-9:00 你的次要活動是什麼(可以不限一個)？",
    "as70f1": "s70f. 學校課後輔導/補習/安親班/上家教",
    "as70f2": "s70f. 寫作業/唸書/準備考試",
    "as70f3": "s70f. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70f4": "s70f. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70f5": "s70f. 吃東西或用餐",
    "as70f6": "s70f. 通勤",
    "as70f7": "s70f. 做家事",
    "as70f8": "s70f. 睡覺",
    "as70f9": "s70f. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70f10": "s70f. 照護/看顧家人",
    "as70f11": "s70f. 與家人/朋友/鄰居聊天",
    "as70f12": "s70f. 照顧寵物/與寵物玩",
    "as70f13": "s70f. 看顧家中的生意/農務/工廠",
    "as70f14": "s70f. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70f15": "s70f. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70f16": "s70f. 在網路上與他人聊天（傳訊息）",
    "as70f17": "s70f. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70f18": "s70f. 在網路上直播",
    "as70f19": "s70f. 線上購物",
    "as70f20": "s70f. 外出遊玩/運動",
    "as70f21": "s70f. 其他（請說明）",
    "aks70f21": "s70f. 其他（請說明）",
    "as70g": "s70g. 晚上9:00-10:00 你的主要活動是什麼？",
    "aks70g": "s70g. 晚上9:00-10:00 你的主要活動是什麼？(21)其他，請說明",
    "as70h": "s70h. 晚上9:00-10:00 你的次要活動是什麼(可以不限一個)？",
    "as70h1": "s70h. 學校課後輔導/補習/安親班/上家教",
    "as70h2": "s70h. 寫作業/唸書/準備考試",
    "as70h3": "s70h. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70h4": "s70h. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70h5": "s70h. 吃東西或用餐",
    "as70h6": "s70h. 通勤",
    "as70h7": "s70h. 做家事",
    "as70h8": "s70h. 睡覺",
    "as70h9": "s70h. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70h10": "s70h. 照護/看顧家人",
    "as70h11": "s70h. 與家人/朋友/鄰居聊天",
    "as70h12": "s70h. 照顧寵物/與寵物玩",
    "as70h13": "s70h. 看顧家中的生意/農務/工廠",
    "as70h14": "s70h. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70h15": "s70h. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70h16": "s70h. 在網路上與他人聊天（傳訊息）",
    "as70h17": "s70h. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70h18": "s70h. 在網路上直播",
    "as70h19": "s70h. 線上購物",
    "as70h20": "s70h. 外出遊玩/運動",
    "as70h21": "s70h. 其他（請說明）",
    "aks70h21": "s70h. 其他（請說明）",
    "as70i": "s70i. 晚上10:00-11:00 你的主要活動是什麼？",
    "aks70i": "s70i. 晚上10:00-11:00 你的主要活動是什麼？(21)其他，請說明",
    "as70j": "s70j. 晚上10:00-11:00 你的次要活動是什麼(可以不限一個)？",
    "as70j1": "s70j. 學校課後輔導/補習/安親班/上家教",
    "as70j2": "s70j. 寫作業/唸書/準備考試",
    "as70j3": "s70j. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70j4": "s70j. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70j5": "s70j. 吃東西或用餐",
    "as70j6": "s70j. 通勤",
    "as70j7": "s70j. 做家事",
    "as70j8": "s70j. 睡覺",
    "as70j9": "s70j. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70j10": "s70j. 照護/看顧家人",
    "as70j11": "s70j. 與家人/朋友/鄰居聊天",
    "as70j12": "s70j. 照顧寵物/與寵物玩",
    "as70j13": "s70j. 看顧家中的生意/農務/工廠",
    "as70j14": "s70j. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70j15": "s70j. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70j16": "s70j. 在網路上與他人聊天（傳訊息）",
    "as70j17": "s70j. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70j18": "s70j. 在網路上直播",
    "as70j19": "s70j. 線上購物",
    "as70j20": "s70j. 外出遊玩/運動",
    "as70j21": "s70j. 其他（請說明）",
    "aks70j21": "s70j. 其他（請說明）",
    "as70k": "s70k. 晚上11:00-12:00 你的主要活動是什麼？",
    "aks70k": "s70k. 晚上11:00-12:00 你的主要活動是什麼？(21)其他，請說明",
    "as70l": "s70l. 晚上11:00-12:00 你的次要活動是什麼（可以不限一個）？",
    "as70l1": "s70l. 學校課後輔導/補習/安親班/上家教",
    "as70l2": "s70l. 寫作業/唸書/準備考試",
    "as70l3": "s70l. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70l4": "s70l. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70l5": "s70l. 吃東西或用餐",
    "as70l6": "s70l. 通勤",
    "as70l7": "s70l. 做家事",
    "as70l8": "s70l. 睡覺",
    "as70l9": "s70l. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70l10": "s70l. 照護/看顧家人",
    "as70l11": "s70l. 與家人/朋友/鄰居聊天",
    "as70l12": "s70l. 照顧寵物/與寵物玩",
    "as70l13": "s70l. 看顧家中的生意/農務/工廠",
    "as70l14": "s70l. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70l15": "s70l. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70l16": "s70l. 在網路上與他人聊天（傳訊息）",
    "as70l17": "s70l. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70l18": "s70l. 在網路上直播",
    "as70l19": "s70l. 線上購物",
    "as70l20": "s70l. 外出遊玩/運動",
    "as70l21": "s70l. 其他（請說明）",
    "aks70l21": "s70l. 其他（請說明）",
    "as70m": "s70m. 晚上12:00-01:00 你的主要活動是什麼？",
    "aks70m": "s70m. 晚上12:00-01:00 你的主要活動是什麼？(21)其他，請說明",
    "as70n": "s70n. 晚上12:00-01:00 你的次要活動是什麼（可以不限一個）？",
    "as70n1": "s70n. 學校課後輔導/補習/安親班/上家教",
    "as70n2": "s70n. 寫作業/唸書/準備考試",
    "as70n3": "s70n. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70n4": "s70n. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70n5": "s70n. 吃東西或用餐",
    "as70n6": "s70n. 通勤",
    "as70n7": "s70n. 做家事",
    "as70n8": "s70n. 睡覺",
    "as70n9": "s70n. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70n10": "s70n. 照護/看顧家人",
    "as70n11": "s70n. 與家人/朋友/鄰居聊天",
    "as70n12": "s70n. 照顧寵物/與寵物玩",
    "as70n13": "s70n. 看顧家中的生意/農務/工廠",
    "as70n14": "s70n. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70n15": "s70n. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70n16": "s70n. 在網路上與他人聊天（傳訊息）",
    "as70n17": "s70n. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70n18": "s70n. 在網路上直播",
    "as70n19": "s70n. 線上購物",
    "as70n20": "s70n. 外出遊玩/運動",
    "as70n21": "s70n. 其他（請說明）",
    "aks70n21": "s70n. 其他（請說明）",
    "as70o": "s70o. 晚上01:00-02:00 你的主要活動是什麼？",
    "aks70o": "s70o. 晚上01:00-02:00 你的主要活動是什麼？(21)其他，請說明",
    "as70p": "s70p. 晚上01:00-02:00 你的次要活動是什麼（可以不限一個）？",
    "as70p1": "s70p. 學校課後輔導/補習/安親班/上家教",
    "as70p2": "s70p. 寫作業/唸書/準備考試",
    "as70p3": "s70p. 上網查學業相關的資料/ 看線上課程/看課外書",
    "as70p4": "s70p. 參加社團活動/ 運動項目練習/才藝活動練習",
    "as70p5": "s70p. 吃東西或用餐",
    "as70p6": "s70p. 通勤",
    "as70p7": "s70p. 做家事",
    "as70p8": "s70p. 睡覺",
    "as70p9": "s70p. 洗澡/刷牙/吹頭髮等個人衛生活動",
    "as70p10": "s70p. 照護/看顧家人",
    "as70p11": "s70p. 與家人/朋友/鄰居聊天",
    "as70p12": "s70p. 照顧寵物/與寵物玩",
    "as70p13": "s70p. 看顧家中的生意/農務/工廠",
    "as70p14": "s70p. 看電視/玩遊戲機（如 PS5, Switch 等）/網路遊戲/手遊",
    "as70p15": "s70p. 上網聽音樂、看影片、動畫、漫畫或有興趣的資訊",
    "as70p16": "s70p. 在網路上與他人聊天（傳訊息）",
    "as70p17": "s70p. 經營自己的社交網站（如剪片、製圖修圖、貼文、上傳照片/影片/音檔）",
    "as70p18": "s70p. 在網路上直播",
    "as70p19": "s70p. 線上購物",
    "as70p20": "s70p. 外出遊玩/運動",
    "as70p21": "s70p. 其他（請說明）",
    "aks70p21": "s70p. 其他（請說明）",
    "as71": "s71. 以下敘述符不符合你的情形？(A卷)",
    "as71a": "s71a. 即使與別人的想法不同，我仍然能夠理解別人的期待是什麼。(A卷)",
    "as71b": "s71b. 我可以和別人心平氣和地討論彼此之間不同的想法。(A卷)",
    "as71c": "s71c. 參考別人的意見來做決定讓我更有信心。(A卷)",
    "as71d": "s71d. 我一向知道自己想要的是什麼。(A卷)",
    "as71e": "s71e. 達成自己想要的目標，對我來說並不是一件困難的事。(A卷)",
    "as71f": "s71f. 對於自己所做的決定，我一向都有信心。(A卷)",
    "as72": "s72. 請勾選在過去1 到2 週您感受到下列選項之頻率。",
    "as72a": "s72a. 我的胃口不好。",
    "as72b": "s72b. 我的睡眠情況不好。",
    "as72c": "s72c. 我的睡眠情況不好。",
    "as72d": "s72d. 我覺得自己是一個壞人。",
    "as72e": "s72e. 我對平時有興趣做的事情都失去興趣。",
    "as72f": "s72f. 我覺得自己的行動太緩慢。",
    "as72g": "s72g. 我總是感到疲憊。",
    "as72h": "s72h. 我無法集中精神做一些重要的事情。",
    "as72i": "s72i. 我覺得坐立不安。",
    "as72j": "s72j. 我覺得很寂寞（孤單、沒伴）。",
    "as72k": "s72k. 我感到緊張焦慮。",
    "as72l": "s72l. 我不能停止或是無法控制擔心。",
    "as73": "s73. 請問過去兩週中，您對以下敘述的感受程度？",
    "as73a": "s73a. 我感到情緒開朗且精神不錯。",
    "as73b": "s73b. 我感到心情平靜和放鬆。",
    "as73c": "s73c. 我感到有活力且精力充沛。",
    "as73d": "s73d. 我醒來感到神清氣爽並有充分休息。",
    "as73e": "s73e. 我的日常生活中充滿讓我感興趣的事物。",
    "as74": "s74. 最近你的身高成長情形怎麼樣?",
    "as75": "s75. 你的體毛（指腋毛和陰毛）生長情形怎麼樣？",
    "as76": "s76. 你注意到任何皮膚改變嗎？（特別是粉刺或青春痘的生長）",
    "as77": "s77. 最近幾個月來，你的體重增加了嗎？ ",
    "as78": "s78. 你開始變聲了嗎？（男生回答，女生請填1）",
    "as79": "s79. 你開始長鬍子了嗎？（男生回答，女生請填1）",
    "as80": "s80. 妳的胸部開始發育了嗎？（女生回答，男生請填1）",
    "as81": "s81. 妳的月經來了嗎？是在什麼時候？（女生回答，男生請填1）",
    "as81a": "s81a. 是在什麼時候？",
    "as82": "s82. 請問你的身高______公分",
    "as83": "s83. 請問你的體重______公斤",
    "as84": "s84. 請勾選出你認為最符合你目前體型的選項:",

    # --- 附加問卷相關 ---
    "attach": "附加問卷. 請問您是否有意願參與附加問卷填寫?",
    "attach2": "附加問卷. 填寫之附加問卷類型",
    "response": "附加問卷. 問卷作答類型",

}
# 您可以在需要時迭代這個字典或查詢特定變項的意義

In [10]:

# --- 如何使用這個 id_map ---
# 假設您已經手動補充完成上面的字典

# 獲取某個變項的說明
variable_id = "as1"
description = id_map.get(variable_id, f"{variable_id} 的說明未在 id_map 中定義")
print(f"變項 {variable_id} 的說明是: {description}")

variable_id = "as9a"
description = id_map.get(variable_id, f"{variable_id} 的說明未在 id_map 中定義")
print(f"變項 {variable_id} 的說明是: {description}")

variable_id = "as28" # 測試一個不存在的變項
description = id_map.get(variable_id, f"{variable_id} 的說明未在 id_map 中定義")
print(f"變項 {variable_id} 的說明是: {description}")


變項 as1 的說明是: s1. 請問你的性別(生理性別)?
變項 as9a 的說明是: s9a. 作決定時,家人會彼此商量。
變項 as28 的說明是: s28. 關於上網，你父親和你有沒有以下的情形？(B卷)


In [11]:
# 通用特殊值定義 (方便複製使用，但通常會直接寫入每個映射字典)
common_special_values = {
    -4: "不適用",
    -5: "不一定",
    -6: "跳答",
    -7: "不知道、不清楚、忘記了",
    -8: "拒答",
    -9: "系統遺漏值",
    -99: "無意義作答/邏輯矛盾",
    -999: "此卷未答"
}

# 學校類型
school_type_value_map = {
    1: "公立",
    2: "私立",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的性別（生理性別）？
as1_value_map = {
    1: "女",
    2: "男",
    # 加入通用特殊值
    **common_special_values
}

# 請問你親生父母的婚姻狀態？
as3_value_map = {
    1: "結婚，且同住一起",
    2: "結婚，因工作分隔兩地",
    3: "結婚，但分居",
    4: "離婚，且分居",
    5: "離婚，但同住一起",
    6: "未婚，但同住一起",
    7: "未婚，且分居",
    8: "親生父親過世",
    9: "結婚，因工作分隔兩地",
    10: "親生父母均過世",
    11: "其他，請說明",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是你的？
as4a1_value_map = {
    1: "父親",
    2: "母親",
    3: "繼父或養父",
    4: "繼母或養母",
    5: "祖父",
    6: "祖母",
    7: "其他", # 對應 (7) 其他，請說明：
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是哪裡人？
as4c1_value_map = {
    1: "臺灣閩南人",
    2: "臺灣客家人",
    3: "大陸各省市(外省人)",
    4: "臺灣原住民",
    5: "大陸籍",
    6: "香港或澳門",
    7: "其他國籍", # 對應 (7) 其他，請說明：
    8: "其他，請說明",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是哪一族？
as4c1_4_value_map = {
    1: "阿美",
    2: "泰雅",
    3: "布農",
    4: "排灣",
    5: "魯凱",
    6: "卑南",
    7: "賽夏",
    8: "鄒",
    9: "邵",
    10: "雅美（達悟）",
    11: "噶瑪蘭",
    12: "太魯閣",
    13: "賽德克",
    14: "撒奇萊雅",
    15: "卡那卡那富",
    16: "拉阿魯哇",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是哪一族？
as4c1_7_value_map = {
    1: "越南",
    2: "印尼",
    3: "菲律賓",
    4: "泰國",
    5: "柬埔寨",
    6: "緬甸",
    7: "馬來西亞",
    8: "日本",
    9: "美國",
    10: "加拿大",
    11: "南韓",
    12: "新加坡",
    13: "其他（請說明）",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者的最高學歷
as4d1_value_map = {
    1: "無／不識字",
    2: "自修／識字",
    3: "小學",
    4: "國（初）中",
    5: "高中職",
    6: "五專、二專",
    7: "大學、二技、四技",
    8: "碩士",
    9: "博士",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者的健康情形
as4e1_value_map = {
    1: "很不健康",
    2: "不健康",
    3: "健康",
    4: "很健康",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第二位主要照顧者是你的？
as4a2_value_map = {
    1: "父親",
    2: "母親",
    3: "繼父或養父",
    4: "繼母或養母",
    5: "祖父",
    6: "祖母",
    7: "其他", # 對應 (7) 其他，請說明：
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是哪裡人？
as4c2_value_map = {
    1: "臺灣閩南人",
    2: "臺灣客家人",
    3: "大陸各省市(外省人)",
    4: "臺灣原住民",
    5: "大陸籍",
    6: "香港或澳門",
    7: "其他國籍", # 對應 (7) 其他，請說明：
    8: "其他，請說明",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是哪一族？
as4c2_4_value_map = {
    1: "阿美",
    2: "泰雅",
    3: "布農",
    4: "排灣",
    5: "魯凱",
    6: "卑南",
    7: "賽夏",
    8: "鄒",
    9: "邵",
    10: "雅美（達悟）",
    11: "噶瑪蘭",
    12: "太魯閣",
    13: "賽德克",
    14: "撒奇萊雅",
    15: "卡那卡那富",
    16: "拉阿魯哇",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者是哪一族？
as4c2_7_value_map = {
    1: "越南",
    2: "印尼",
    3: "菲律賓",
    4: "泰國",
    5: "柬埔寨",
    6: "緬甸",
    7: "馬來西亞",
    8: "日本",
    9: "美國",
    10: "加拿大",
    11: "南韓",
    12: "新加坡",
    13: "其他（請說明）",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者的最高學歷
as4d2_value_map = {
    1: "無／不識字",
    2: "自修／識字",
    3: "小學",
    4: "國（初）中",
    5: "高中職",
    6: "五專、二專",
    7: "大學、二技、四技",
    8: "碩士",
    9: "博士",
    # 加入通用特殊值
    **common_special_values
}

# 請問你的第一位主要照顧者的健康情形
as4e2_value_map = {
    1: "很不健康",
    2: "不健康",
    3: "健康",
    4: "很健康",
    # 加入通用特殊值
    **common_special_values
}

# 請問你是否有兄弟姐妹？
as5_value_map = {
    1: "有",
    2: "沒有",
    # 加入通用特殊值
    **common_special_values
}

# 那你有幾個兄弟姐妹
as6_value_map = {
    0: "0 人 ",
    1: "1 人",
    2: "2 人",
    3: "3 人",
    4: "4 人",
    5: "5 人以上",
    # 加入通用特殊值
    **common_special_values
}
as6a_value_map = as6b_value_map = as6c_value_map = as6d_value_map = as6_value_map

accord_value_map = {
    1: "很不符合",
    2: "不符合",
    3: "符合",
    4: "很符合",
    # 加入通用特殊值
    **common_special_values
}
as7a_value_map = as7b_value_map = as7c_value_map = accord_value_map

binary_value_map = {
    0: "沒有",
    1: "有",
    # 加入通用特殊值
    **common_special_values
}

as8_1_value_map = as8_2_value_map = as8_3_value_map = as8_4_value_map = as8_5_value_map = binary_value_map
as8_6_value_map = as8_7_value_map = as8_8_value_map = as8_9_value_map = as8_10_value_map = binary_value_map
as8_11_value_map = as8_12_value_map = as8_13_value_map = as8_14_value_map = as8_15_value_map = binary_value_map
as8_16_value_map = as8_17_value_map = as8_18_value_map = as8_19_value_map = as8_20_value_map = binary_value_map
as8_21_value_map = as8_22_value_map = as8_23_value_map = binary_value_map

as9a_value_map = as9b_value_map = as9v_value_map = as9d_value_map = as9e_value_map = as9f_value_map = accord_value_map

as10_value_map = {
    1: "很不富裕",
    2: "不富裕",
    3: "富裕",
    4: "很富裕",
    # 加入通用特殊值
    **common_special_values
}

as11_value_map = {
    1: "沒有宗教信仰",
    2: "民間信仰",
    3: "佛教",
    4: "道教",
    5: "一貫道",
    6: "天主教",
    7: "基督教：______教派",
    8: "回教",
    9: "其他，請說明：______",
    # 加入通用特殊值
    **common_special_values
}

time_frequency_value_map = {
    1: "從未",
    2: "偶爾",
    3: "有時",
    4: "經常",
    # 加入通用特殊值
    **common_special_values
}

as12a_value_map = as12b_value_map = as12c_value_map = as12d_value_map = as12e_value_map = as12f_value_map = time_frequency_value_map
as12g_value_map = as12h_value_map = as12i_value_map = as12j_value_map = time_frequency_value_map

satisfy_value_map = {
    1: "很不滿意 ",
    2: "不滿意",
    3: "滿意",
    4: "很滿意",
    # 加入通用特殊值
    **common_special_values
}

as13_value_map = satisfy_value_map

agree_value_map = {
    1: "很不同意",
    2: "不同意",
    3: "同意",
    4: "很同意",
    # 加入通用特殊值
    **common_special_values
}

as14a_value_map = as14b_value_map = as14c_value_map = agree_value_map

as15a_value_map = as15b_value_map = accord_value_map

as16a_value_map = as16b_value_map = as16c_value_map = as16d_value_map = as16e_value_map = accord_value_map

In [14]:

# --- 步驟 2: 應用映射，創建帶有文字標籤的新欄位 ---
# 建議將轉換後的標籤存儲在新欄位中，以便保留原始數值欄位

# 使用 .map() 方法，它會自動將字典中未定義的 Key 映射為 NaN
# 檢查欄位是否存在是一個好習慣
if "as4a1" in df.columns:
    df["as4a1_label"] = df["as4a1"].map(as4a1_value_map)
else:
    print("警告：欄位 as4a1 不存在")

if "as1" in df.columns:
    df["as1_label"] = df["as1"].map(as1_value_map)
else:
    print("警告：欄位 as1 不存在")

if "as9a" in df.columns:
    df["as9a_label"] = df["as9a"].map(as9a_value_map)
else:
    print("警告：欄位 as9a 不存在")

if "as5" in df.columns:
    df["as5_label"] = df["as5"].map(as5_value_map)
else:
    print("警告：欄位 as5 不存在")

# --- 步驟 3: 檢視結果 ---
# 選擇要顯示的欄位，只顯示存在的欄位
display_cols = []
for var in ["as1", "as5", "as9a", "as4a1"]:
    if var in df.columns:
        display_cols.extend([var, f"{var}_label"]) # 加入原始值和標籤欄位

if display_cols:
    print("\n轉換結果預覽 (部分欄位，前 5 行):")
    print(df[display_cols].head())
else:
    print("\n所有範例欄位都不在 DataFrame 中，無法顯示預覽。")


# --- 如何處理 "其他(請說明)" ---
# 對於像 as4a1 中數值 7 代表 "其他" 的情況，其對應的文字說明通常儲存在 aks4a1 欄位中。
# 您可以使用轉換後的 as4a1_label 來輔助查看 aks4a1 的內容：
# print(df.loc[df["as4a1_label"] == "其他", ["as4a1", "as4a1_label", "aks4a1"]].head())


轉換結果預覽 (部分欄位，前 5 行):
   as1 as1_label  as5 as5_label  as9a as9a_label  as4a1 as4a1_label
0    2         男    1         有     3         符合      1          父親
1    2         男    1         有     3         符合      1          父親
2    2         男    1         有     3         符合      2          母親
3    2         男    2        沒有     4        很符合      7          其他
4    2         男    1         有     4        很符合      2          母親
